# Extracción de datos Datasets Google Maps y YELP


### Importaciones


In [1]:
import pandas as pd
import numpy as np
import json
import datetime
import os

### Definiciones y carpetas


Creación de diccionario de Estados


In [2]:
state_abreviations = [
    "AL",
    "AK",
    "AZ",
    "AR",
    "CA",
    "CO",
    "CT",
    "DE",
    "FL",
    "GA",
    "HI",
    "ID",
    "IL",
    "IN",
    "IA",
    "KS",
    "KY",
    "LA",
    "ME",
    "MD",
    "MA",
    "MI",
    "MN",
    "MS",
    "MO",
    "MT",
    "NE",
    "NV",
    "NH",
    "NJ",
    "NM",
    "NY",
    "NC",
    "ND",
    "OH",
    "OK",
    "OR",
    "PA",
    "RI",
    "SC",
    "SD",
    "TN",
    "TX",
    "UT",
    "VT",
    "VA",
    "WA",
    "WV",
    "WI",
    "WY",
]

state_dictionary = {
    "AL": "Alabama",
    "AK": "Alaska",
    "AZ": "Arizona",
    "AR": "Arkansas",
    "CA": "California",
    "CO": "Colorado",
    "CT": "Connecticut",
    "DE": "Delaware",
    "FL": "Florida",
    "GA": "Georgia",
    "HI": "Hawaii",
    "ID": "Idaho",
    "IL": "Illinois",
    "IN": "Indiana",
    "IA": "Iowa",
    "KS": "Kansas",
    "KY": "Kentucky",
    "LA": "Louisiana",
    "ME": "Maine",
    "MD": "Maryland",
    "MA": "Massachusetts",
    "MI": "Michigan",
    "MN": "Minnesota",
    "MS": "Mississippi",
    "MO": "Missouri",
    "MT": "Montana",
    "NE": "Nebraska",
    "NV": "Nevada",
    "NH": "New Hampshire",
    "NJ": "New Jersey",
    "NM": "New Mexico",
    "NY": "New York",
    "NC": "North Carolina",
    "ND": "North Dakota",
    "OH": "Ohio",
    "OK": "Oklahoma",
    "OR": "Oregon",
    "PA": "Pennsylvania",
    "RI": "Rhode Island",
    "SC": "South Carolina",
    "SD": "South Dakota",
    "TN": "Tennessee",
    "TX": "Texas",
    "UT": "Utah",
    "VT": "Vermont",
    "VA": "Virginia",
    "WA": "Washington",
    "WV": "West Virginia",
    "WI": "Wisconsin",
    "WY": "Wyoming",
}

Creación de directorios si no existen


In [3]:
generated_dir = "Generated"

if not os.path.exists(generated_dir):
    os.mkdir(generated_dir)

os.chdir(generated_dir)

if not os.path.exists("Google"):
    os.mkdir("Google")
if not os.path.exists("Yelp"):
    os.mkdir("Yelp")

os.chdir("../")

# 1. Extracción con los datasets de Google Maps


## 1.1 Metada de Sitios


Se recorren los archivos línea por línea, durante el proceso se almacenan sólo las filas que incluyan <code>Restaurant</code> en la columna de categoría, así se optimiza el tamaño del dataset final.


In [4]:
lineas_json = []

for i in range(1, 12):
    path = f"Datasets/Google Maps/metadata-sitios/{i}.json"
    with open(path, "r") as file:
        for l in file:
            try:
                linea_j = json.loads(l)
                if "restaurant" in " ".join(linea_j["category"]).lower():
                    lineas_json.append(linea_j)
            except:
                pass

df = pd.DataFrame(lineas_json)

df.to_parquet(r"Generated\Google\metada_sitios.parquet")

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212014 entries, 0 to 212013
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   name              212014 non-null  object 
 1   address           210956 non-null  object 
 2   gmap_id           212014 non-null  object 
 3   description       72392 non-null   object 
 4   latitude          212014 non-null  float64
 5   longitude         212014 non-null  float64
 6   category          212014 non-null  object 
 7   avg_rating        212014 non-null  float64
 8   num_of_reviews    212014 non-null  int64  
 9   price             100000 non-null  object 
 10  hours             189913 non-null  object 
 11  MISC              210599 non-null  object 
 12  state             190751 non-null  object 
 13  relative_results  170957 non-null  object 
 14  url               212014 non-null  object 
dtypes: float64(3), int64(1), object(11)
memory usage: 24.3+ MB


Tamaño Directorio <code>metadata-sitios</code>: 2.76 Gb

Tamaño Archivo <code>metada_sitios.parquet</code>: 60.43 Mb

Dimensiones: 212.014 filas x 15 Columnas

## 1.1.1 Obtención de información de Estados


En base al campo <code>Address</code> obtenemos el estado donde se encuentra el negocio. Nos servirá para luego seleccionar los estados con más restaurantes.


In [5]:
def get_state_ab(st):
    try:
        state = st.split(", ")[-1].split(" ")[0]
        if state in state_abreviations:
            return state
        else:
            return np.nan
    except:
        return np.nan

df["state_ab"] = df["address"].apply(get_state_ab)

#Obtención Top 5 estados con más restaurantes
top_5 = df["state_ab"].value_counts().head(5).index.to_list()
#top_5
df["us_state"] = df["state_ab"].map(state_dictionary)

#Arreglo de URLs archivos correspondientes
top_5_url = [
    f"Datasets/Google Maps/reviews-estados/review-{state_dictionary[i].replace(' ', '_')}/"
    for i in top_5
]

#Creación diccionario cantidad de archivos
cantidad_archivos = {}

for i in top_5_url:
    for j in os.walk(i):
        cantidad_archivos[i] = len(j[2])



## 1.2 Reviews Estados


Ya con los estados elegidos estamos en condiciones de ingestar los datos de las carpetas correspondientes dentro del directorio <code>reviews-estados</code>.
Es información masiva lo que genera un archivo de grandes dimensiones, sin embargo previamente filtramos por el parámetro de año <code>2017-2019</code> valiéndonos del campo <code>time</code>, que tiene es un <code>timestamp</code>, pero con 3 digitos más que el usado por <code>datetime</code> de Python. Le agregamos el campo <code>Estado</code> que es más descriptivo.


In [7]:
lineas_json_revs_google = []

for i in top_5_url:
    count = 0
    for c in range(1, cantidad_archivos[i] + 1):
        with open(str(i) + str(c) + ".json", "r", encoding="utf-8") as f:
            for s in f:
                linea = json.loads(s)
                linea["anio"] = datetime.datetime.fromtimestamp(
                    linea["time"] / 1000
                ).year
                linea["estado"] = i.split("-")[-1][:-1]

                if linea["anio"] in [2017, 2018, 2019]:
                    lineas_json_revs_google.append(linea)

df_revs_google = pd.DataFrame(lineas_json_revs_google)

merge_site_reviews = pd.merge(df_revs_google, df, left_on="gmap_id", right_on="gmap_id")

merge_site_reviews.to_parquet(r"Generated\Google\merge_site_reviews.parquet")

df_revs_google.to_parquet(r"Generated\Google\reviews-estados.parquet")

df_revs_google.info()

Tamaño archivo: 760 Mb

Tamaño dataset: 24.3 Gb

Tamaño 8.339.179 filas x 10 Columnas.


# 2. Extracción de los Dataset de YELP


### 2.1 Business


Contiene los datos de las entidades negocios de Yelp, a un primer vistazo tiene las columnas duplicadas, por lo que hay que hacer un recorte, ya que la segunda mitad tiene datos vacíos en su inmensa mayoría.


In [ ]:
url_business = r"Datasets\Yelp\business.pkl"
df_business = pd.read_pickle(url_business)
df_business = df_business.iloc[:, :-14]

# Filtrado de negocios según el campo "state"

df_business = df_business[df_business.state.isin(top_5)]

# Filtrado de negocios según el campo "categories"

def is_restaurant(st):
    try:
        test = "".join(st).lower()
        return "restaurant" in test
    except:
        return False
    
df_business = df_business[df_business["categories"].apply(is_restaurant)]

df_business.to_parquet(r"Generated\Yelp\bussines.parquet")

### 2.2 Checkin


Se decidió que los datos de registros en el negocio por parte de los usuarios no era información pertinente para el proyecto, por lo que el archivo "checkin.json" no será cargado

### 2.3 Tips


Se decidió que los datos de consejos de los usuarios no era información pertinente para el proyecto, por lo que el archivo "tip.json" no será cargado

### 2.4 Review


In [ ]:
df_reviews_url = r"Datasets\Yelp\review.json"

# Extracción Linea por línea y filtrdo por año y "useful"

lineas_json_review = []

with open(df_reviews_url, "r", encoding="utf-8") as f:
    count = 0
    for i in f:
        linea = json.loads(i)
        anio = linea["date"][:4]
        if anio in ["2017", "2018", "2019"] and linea["useful"] == 1:
            lineas_json_review.append(linea)

df_reviews = pd.DataFrame(lineas_json_review)

# Downgrade de tipo de variable para disminuir tamaño
df_reviews["funny"] = df_reviews["funny"].astype("int8")
df_reviews["stars"] = df_reviews["stars"].astype("int8")
df_reviews["cool"] = df_reviews["cool"].astype("int8")

df_reviews.drop("useful", axis=1, inplace=True, errors="ignore")

df_reviews.to_parquet(r"Generated\Yelp\review.parquet")

### 2.4 Users Yelp


Se decidió que los datos directamente relacionados a los usuarios no era información pertinente para el proyecto, por lo que el archivo "users.parquet" no será cargado